In [24]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import time
import datetime

In [25]:
dataset_dir = "dataset/"

filenames = os.listdir(dataset_dir)

print(filenames)

['video1.mp4', 'video2.mp4']


**1. Dasar membaca dan menampilkan gambar pada OpenCV**

In [27]:
img = cv2.imread('sample/blocks.jpg')

cv2.imshow('blocks',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

**2. Dasar membaca dan menampikan video pada OpenCV**

In [28]:
cap = cv2.VideoCapture('dataset/video1.mp4')

while cap.isOpened():
    ret, frame = cap.read()
    
    if ret:
        frame = cv2.resize(frame, (0,0), fx=0.5, fy=0.5)
        cv2.imshow('video 1', frame)
        
        if cv2.waitKey(25) == ord('q'):
            break
    else :
        break
        
cap.release()
cv2.destroyAllWindows() 

**3. Dasar Image Masking menggunakan bitwise_and**

In [29]:
img = cv2.imread('sample/lena.jpg')

mask = np.zeros_like(img[:,:,0])
h, w = mask.shape

cv2.circle(mask, (h//2, w//2), 150, (255, 255, 255), -1)

res = cv2.bitwise_and(img, img, mask=mask)

cv2.imshow('frame', img)
cv2.imshow('res',res)
cv2.imshow('mask',mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [30]:
lower = np.array([110, 50, 50])
upper = np.array([130, 255, 255])

img = cv2.imread('sample/blocks.jpg')

hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
mask = cv2.inRange(hsv.copy(), lower, upper)
res = cv2.bitwise_and(img, img, mask=mask)

cv2.imshow('frame',img)
cv2.imshow('res',res)
cv2.imshow('mask',mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
filenames[0]

'video1.mp4'

In [9]:
img.shape

(360, 480, 3)

**Shape based (Hough Circle Transform)**

In [23]:
img = cv2.imread('sample/eye.jpg')
h, w, c = img.shape

gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, h/64, param1=200, param2=17, minRadius=21, maxRadius=30)

if circles is not None:
    circles = np.uint16(np.around(circles))[0]
    for i in circles:
        cv2.circle(img, (i[0], i[1]), i[2], (0, 255, 0), 2)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(res, 'Ball', (10, 300), font, 2, (0,0,255), 1, cv2.LINE_AA)
        
cv2.imshow("output", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [31]:
# Fungsi deteksi warna
def detectColor(img, lower, upper):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv.copy(), lower, upper)
    res = cv2.bitwise_and(img, img, mask=mask)  #bitwise opration
    return res


In [32]:
# Fungsi membuat trace pada contour
def findContourCircle(res):
        x, y, r = 0, 0, 0
        gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
        contours, hierarchy = cv2.findContours(gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        circle = []
        for cnt in contours:
            (x,y), radius = cv2.rectangle(cnt)
            
            if radius > 10: 
                circle.append([int(x), int(y), int(radius)])
        
        if len(circle) > 0 :
            circle = np.array(circle)
            x = int(np.mean(circle[:, 0]))
            y = int(np.mean(circle[:, 1]))
            r = int(np.mean(circle[:, 2]))
            
        return x, y, r

In [33]:
# Dictonary penyimpanan warna
color_ranges = dict(
    blue = dict(
        lower = np.array([110, 50, 50]),
        upper = np.array([130, 255, 255])
    ),
    orange = dict(
        lower = np.array([10, 50, 50]),
        upper = np.array([20, 255, 255])
    )
)

availabel_colors = color_ranges.keys()

In [34]:
# Extraksi bentuk lingkaran yang tidak sempurna
def findCircelHoug(img):
    rs = []
    x, y, r = 0, 0, 0
    h, w , c = img.shape
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ret, binary = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV)
    
    edge = cv2.Canny(binary, 100, 200)
    
    binary = cv2.resize(binary, (0,0), fx=1, fy=1)
    cv2.imshow("binary", binary)
    
    circles = cv2.HoughCircles(edge, cv2.HOUGH_GRADIENT, 1, h/64, param1=150, param2=20, minRadius=50, maxRadius=100)
        
    if circles is not None:
        circles = np.uint16(np.around(circles))[0]
        x = int(np.mean(circles[:, 0]))
        y = int(np.mean(circles[:, 1]))
        rs.append(np.mean(circles[:, 2]))
        r = int(np.mean(rs))
        
    return x, y, r

**Deteksi Objek** <br/> 
**Konsep 1. Color Based (range thresholding)** <br/>
**Konsep 2. Shape Based (hough circle transform)** <br/>

In [36]:
color_input = input("pilih warna [blue/orange] :")

while color_input not in availabel_colors:
    color_input = input("%s tidak ditemukan, pilih warna [blue/orange] :" % color_input)

# filename = filenames[0]
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, img = cap.read()
    
    if ret:
        color = color_ranges[color_input]
        res = detectColor(img, color['lower'], color['upper'])
        
        x, y, r = findCircelHoug(res)
        cv2.circle(img, (x, y), r, (255, 0, 255), 2)
        
        res = cv2.resize(res, (0,0), fx=1, fy=1)
        img = cv2.resize(img, (0,0), fx=1, fy=1)
        
        cv2.imshow(filename + " - detected", res)
        cv2.imshow(filename, img)
        
        if cv2.waitKey(25) == ord('q'):
            break
    else :
        break
        
cap.release()
cv2.destroyAllWindows()

pilih warna [blue/orange] :blue
